In [1]:
import gzip
import json

import MeCab
import unidic

import pickle
import time

import math
from tqdm import tqdm

from collections import Counter, defaultdict

In [2]:
def load_index_line(index_line):
    return list(map(lambda x: tuple(map(int, x.split(':'))), index_line.split(' ')))

In [3]:
# def search_entity(query, topk=13):
def search_entity(query, topk=10):
    avgdl = sum(doc_id2token_count) / len(doc_id2token_count)
    parsed_query = parse_text(query)
    target_posting = {}
    with open('../ir_dump/inverted_index', 'r', encoding='utf-8') as index_file:
        for token in parsed_query:
            if token in token2pointer:
                pointer, offset = token2pointer[token]
                index_file.seek(pointer)
                index_line = index_file.read(offset-pointer).rstrip()
                postings_list = load_index_line(index_line)
                target_posting[token] = postings_list

    # bm25スコアでor検索
    k1 = 2.0
    b = 0.75
    all_docs = len(entities)
    doc_id2tfidf = [0 for i in range(all_docs)]
    for token, postings_list in target_posting.items():
        idf = math.log2((all_docs-len(postings_list)+0.5) / (len(postings_list) + 0.5))
        # idfが負になる単語は一般的すぎるので無視
        idf = max(idf, 0)
        if idf == 0:
            continue
        for doc_id, tf in postings_list:
            dl = doc_id2token_count[doc_id]
            token_tfidf = idf * ((tf * (k1 + 1))/(tf + k1 * (1-b+b*(dl/avgdl))))
            doc_id2tfidf[doc_id] += token_tfidf

    docs = [(doc_id, tfidf) for doc_id, tfidf in enumerate(doc_id2tfidf) if tfidf != 0]
    docs = sorted(docs, key=lambda x: x[1], reverse=True)
    return docs[:topk]

In [4]:
def get_sentence_list(search_result, topk=5):
    sentence_list = []
    for doc_id, _ in search_result[:topk]:
        title = doc_id2title[doc_id]
        text = entities[title]
        sentences = text.split('。')
        sentence_list += sentences
    return sentence_list

In [5]:
input_file = '../data/all_entities.json.gz'
with gzip.open(input_file, "rt", encoding="utf-8") as fin:
    lines = fin.readlines()

In [6]:
entities = dict()
for line in lines:
    entity = json.loads(line.strip())
    entities[entity["title"]] = entity["text"]
del lines

In [7]:
tagger = MeCab.Tagger('-d "{}"'.format(unidic.DICDIR))
STOP_POSTAGS = ('代名詞', '接続詞', '感動詞', '動詞,非自立可能', '助動詞', '助詞', '接頭辞', '記号,一般', '補助記号', '空白', 'BOS/EOS')

In [8]:
def parse_text(text):
    node = tagger.parseToNode(text)
    tokens = []
    while node:
        if node.feature.startswith(STOP_POSTAGS):
            pass
        elif len(feature := node.feature.split(",")) > 7:            
            tokens += [feature[7].lower()]
        else:
            tokens += [node.surface.lower()]
        node = node.next
    return tokens

In [9]:
def sort_sentence(query, sentence_list, topk=10):
    inverted_index = defaultdict(list)
    sentence_id2sentence = [sentence for sentence in sentence_list]
    sentence_id2token_count = []
#     for sentence_id, sentence in tqdm(enumerate(sentence_list), total=len(sentence_list)):
    for sentence_id, sentence in enumerate(sentence_list):
        tokens = parse_text(sentence)
    
        sentence_id2token_count += [len(tokens)]

        count_tokens = Counter(tokens)
        for token, count in count_tokens.items():
            inverted_index[token] += [(sentence_id, count)]

    avgdl = sum(sentence_id2token_count) / len(sentence_id2token_count)
    parsed_query = parse_text(query)
    target_posting = {}
    for token in parsed_query:
        if token in inverted_index:
            postings_list = inverted_index[token]
            target_posting[token] = postings_list

    # bm25スコアでor検索
    k1 = 2.0
    b = 0.75
    all_docs = len(sentence_list)
    sentence_id2tfidf = [0 for i in range(all_docs)]
    for token, postings_list in target_posting.items():
        idf = math.log2((all_docs-len(postings_list)+0.5) / (len(postings_list) + 0.5))
        # idfが負になる単語は一般的すぎるので無視
        idf = max(idf, 0)
        if idf == 0:
            continue
        for sentence_id, tf in postings_list:
            dl = sentence_id2token_count[sentence_id]
            token_tfidf = idf * ((tf * (k1 + 1))/(tf + k1 * (1-b+b*(dl/avgdl))))
            sentence_id2tfidf[sentence_id] += token_tfidf

    sentences = [(sentence_id, tfidf) for sentence_id, tfidf in enumerate(sentence_id2tfidf) if tfidf != 0]
    sentences = sorted(sentences, key=lambda x: x[1], reverse=True)
    return list(map(lambda x: (sentence_id2sentence[x[0]], x[1]), sentences[:topk]))

In [10]:
input_file = '../data/train_questions.json'
with open(input_file, "r", encoding="utf-8") as fin:
    lines = fin.readlines()

queries = []
for line in tqdm(lines):
    data_raw = json.loads(line.strip("\n"))
#     qid = data_raw["qid"]
    question = data_raw["question"].replace("_", "")  # "_" は cloze question
#     options = data_raw["answer_candidates"]  # TODO
#     answer = data_raw["answer_entity"]
    queries += [question]

100%|██████████| 13061/13061 [00:00<00:00, 119165.75it/s]


In [11]:
with open('../ir_dump/doc_id2title.pickle', 'rb') as f:
    doc_id2title = pickle.load(f)
with open('../ir_dump/doc_id2token_count.pickle', 'rb') as f:
    doc_id2token_count = pickle.load(f)
with open('../ir_dump/token2pointer.pickle', 'rb') as f:
    token2pointer = pickle.load(f)

In [12]:
%%time
result = search_entity(queries[0], topk=1)

CPU times: user 1.01 s, sys: 20 ms, total: 1.03 s
Wall time: 1.03 s


In [13]:
%%time
# sentence_list = entities[doc_id2title[result[0][0]]].split('。')
sentence_list = get_sentence_list(result)
sorted_sentence = sort_sentence(queries[0], sentence_list)

CPU times: user 27.3 ms, sys: 3.96 ms, total: 31.2 ms
Wall time: 31.2 ms


In [14]:
sorted_sentence

[('ボブ・サップ（Bob Sapp、1973年9月22日 - ）は、アメリカ合衆国のキックボクサー、プロレスラー、総合格闘家、タレント、俳優、元アメリカンフットボール選手',
  18.001643434637835),
 ('彼とは別にアメリカ出身で同姓同名の格闘家・マイケル・マクドナルドが存在するが、こちらは1991年生まれでありUFCを中心として総合格闘技一本に絞った競技生活を送っている',
  13.911810695403958),
 (' ^ Bob Sapp explains DREAM "Dynamite!! 2010" no-show, says DREAM is "broke" MMA JUNKIE.COM 2011年1月18日 ^ SimonInoki 2011年1月17日 ^ 野獣ボブ・サップ怒りの暴露「格闘技界のカネ、女、ＦＢＩ」 vol.1デジタル大衆 週刊大衆4月1日号 ^ 野獣ボブ・サップ怒りの暴露「格闘技界のカネ、女、ＦＢＩ」 vol.2 デジタル大衆 週刊大衆4月1日号 ^ カナダ出身の格闘家で、1965年生まれ',
  11.611478655388193),
 ('なお、ウォーレン・サップはフロリダ州オーランド出身であり、マイアミ大学の卒業生である', 9.802641668346661),
 (' 2010年、格闘家として第一線を退き、韓国でトークショーの司会や、会社の経営に乗り出していると明言', 9.05018428626646),
 ('コロラド州コロラドスプリングス出身', 8.847691695396007),
 (' 2002年、同じく元WCWの選手で友人のサム・グレコの紹介でK-1にスカウトされ、格闘家としてPRIDE、K-1に参戦',
  8.163112398717395),
 (" 男子総合格闘家一覧 男子キックボクサー一覧 プロレスラー一覧 DREAM選手一覧 PRIDE選手一覧 HERO'S選手一覧 K-1選手一覧 ニュースタッフプロダクション ボブ・サップ オフィシャルサイト ボブ・サップ オフィシャルブログ Bob Sapp ボブ サップ (@bobsappmma) - Twitter ボブ サップ - Facebook BobSappTV's channel - 公式YouTubeチャンネル 